# Lab 1: Information Retrieval

__Students:__ 

### Crawling



a) Get the webpage content by using functions in 
__[urllib module](https://docs.python.org/3/library/urllib.html#module-urllib)__.

Other libraries are also fine to achieve the crawling.

e.g. scrapy, beautifulsoup... 

In [1]:
import urllib.request
import re


x = urllib.request.urlopen('https://play.google.com/store/apps/category/GAME?hl=en').read().decode('utf-8')


In [2]:
# Get all games from start page
appreg = r'href=\"(/store/apps/details.*?)\"'
appre = re.compile(appreg)
app_url_list = re.findall(appre,x)

# Get all game categories
reg1 = r'href=\"(/store/apps/category/GAME_.*?)\"'
cat_re = re.compile(reg1)
cat_url_list = re.findall(cat_re,x)
print(cat_url_list)

['/store/apps/category/GAME_ACTION', '/store/apps/category/GAME_ADVENTURE', '/store/apps/category/GAME_ARCADE', '/store/apps/category/GAME_BOARD', '/store/apps/category/GAME_CARD', '/store/apps/category/GAME_CASINO', '/store/apps/category/GAME_CASUAL', '/store/apps/category/GAME_EDUCATIONAL', '/store/apps/category/GAME_MUSIC', '/store/apps/category/GAME_PUZZLE', '/store/apps/category/GAME_RACING', '/store/apps/category/GAME_ROLE_PLAYING', '/store/apps/category/GAME_SIMULATION', '/store/apps/category/GAME_SPORTS', '/store/apps/category/GAME_STRATEGY', '/store/apps/category/GAME_TRIVIA', '/store/apps/category/GAME_WORD', '/store/apps/category/GAME_ACTION', '/store/apps/category/GAME_ADVENTURE', '/store/apps/category/GAME_ARCADE', '/store/apps/category/GAME_BOARD', '/store/apps/category/GAME_CARD', '/store/apps/category/GAME_CASINO', '/store/apps/category/GAME_CASUAL', '/store/apps/category/GAME_EDUCATIONAL', '/store/apps/category/GAME_MUSIC', '/store/apps/category/GAME_PUZZLE', '/store/a

b) Get app url by regular expression using functions from __[re module](https://docs.python.org/3/library/re.html?highlight=re#module-re)__.

A useful online regular expression check.
__[Check your regular expression first](https://regex101.com)__.

In [3]:
start = 'https://play.google.com'
url_ending = '/collection/topselling_new_free?hl=en'
url_ending2 = '/collection/topgrossing?hl=en'
# Get games from all categories, use topselling and topgrossing for more results
for e in cat_url_list:
    x = urllib.request.urlopen(start+e+url_ending).read().decode('utf-8')
    app_url_list += re.findall(appre,x)
    x = urllib.request.urlopen(start+e+url_ending2).read().decode('utf-8')
    app_url_list += re.findall(appre,x)
app_url_list = list(set(app_url_list))
print(len(app_url_list))
print(app_url_list[:10])

1712
['/store/apps/details?id=com.VegasCasinoGames.ChineseSlots', '/store/apps/details?id=com.kongregate.mobile.cobra.google', '/store/apps/details?id=com.gameinsight.tribez', '/store/apps/details?id=com.zitga.ninja.stickman.legends.shadow.wars', '/store/apps/details?id=com.gameinsight.airport', '/store/apps/details?id=pl.com.berobasket.speedwaychallengecareer.android', '/store/apps/details?id=dja.bestapp.jojosiwaa.piano.pro', '/store/apps/details?id=com.teamlava.bubble', '/store/apps/details?id=com.playrix.gardenscapes', '/store/apps/details?id=com.MileStoneStudios.SuperheroFlash.multispeed.LightningSpeedster.hero']


c) Access specific webpage to get description of each app and then store the description in files.

In [4]:
# Regex to extract app descriptions
desc_re = r'itemprop=\"description.*?\">.*?<div jsname=\".*?\">.*?</div>'
desc_reg = re.compile(desc_re)
descriptions = []

# Get raw description text
for url_ in app_url_list:
    x = urllib.request.urlopen(start+url_+'&hl=en').read().decode('utf-8')
    descriptions += re.findall(desc_reg,x)
print(descriptions[4])

itemprop="description"><content>• The mischievous Pixies are catching butterflies! Take part in a new event, help the butterflies return to the Sakura, and they’ll lead you to great rewards! Collect as many cherries as possible and get new decorations!<br>• New exciting mini-game! The branches of the Sakura hold many resources: clear them to reach the most valuable reward!<br>• Don&#39;t miss the epic Black Friday event at the end of November! Unbelievable prices and unique offers await everyone!</content><div jsname="WgKync" class="uwAgLc"></div>


In [5]:
get_text_re = r'(?<=\<content\>)(.*)(?=\</content)'
get_text_reg = re.compile(get_text_re)
desc_texts = []
# Remove html-tags from the text
for desc in descriptions:
    tmp = re.findall(get_text_re,desc)
    if tmp != []:
        text = tmp[0].replace('<br>',' ')
        desc_texts += [text]
        
print(desc_texts[22])

New release


In [6]:
import numpy as np

np.save('descriptions',desc_texts)

In [7]:
desc_texts = np.load('descriptions.npy')

### Construct Inverted file index (Vector Model)



d) Preprocess text using NLP techniques from __[nltk module](http://www.nltk.org/py-modindex.html)__ or spaCy.

Using nltk.download(ID) to get the corpora if it is not downloaded before. __[nltk corpora](http://www.nltk.org/nltk_data/)__

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/johmy592/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
import string
# Romove non alphanumerical characters, and fix ' signs
for i in range(len(desc_texts)):
    desc_texts[i] = ''.join(ch.lower() for ch in desc_texts[i] if ch.isalnum() or ch==' ')
    desc_texts[i] = desc_texts[i].replace('39','\'')
    
print(desc_texts[22])

new release


In [10]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()
# Tokenize, lemmatize and remove stopwords
tokens = [word_tokenize(t) for t in desc_texts]
tokens = [[wnl.lemmatize(t) for t in d if t not in stopwords.words('english')] for d in tokens]
print(tokens[6])

['700', 'level', 'discover', 'exciting', 'new', 'level', 'across', 'ton', 'beautiful', 'island', 'new', 'weekly', 'reward', 'collect', 'puzzle', 'piece', 'level', 'win', 'weekly', 'prize', 'new', 'leaderboard', 'compete', 'win', 'prize', 'based', 'weekly', 'rank']


...)Compute tdidf 
eg. Using functions from __[scikit-learn module](http://scikit-learn.org/stable/modules/classes.html)__. TfidfVectorizer is used for converting a collection of raw documents to a matrix of TF-IDF features.
#### You can also build the tfidf matrix with other library or your own algorithm

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
transvector = TfidfVectorizer()
corpus = [
     'This is the first document.',
     'This is the second second document.',
     'And this is the third one.',
     'Is this the first document?',]
tfidf1 = transvector.fit_transform(corpus)
print(tfidf1.toarray())
print(transvector.get_feature_names())


[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.27925389 0.         0.22830836 0.         0.87501037
  0.22830836 0.         0.22830836]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [12]:
# Create a Tfidf model using sklearns built in model
# Overload preprocessor and tokenizer since we have already done this
transvector = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
tfidf1 = transvector.fit_transform(tokens)

In [13]:
import scipy as sp
from scipy.sparse import csr_matrix

### Query Process

eg. "Dragon, Control, hero, running"

eg. "The hero controls the dragon to run."



In [14]:

def preprocess_search(search_term):
    '''
    Preprocess search term in the same way as the descriptions before
    '''
    tolower = ''.join(ch.lower() for ch in search_term if ch.isalnum() or ch==' ')
    tok = word_tokenize(tolower) 
    return [wnl.lemmatize(t) for t in tok if t not in stopwords.words('english')]

def get_top_results(search_term,k):
    '''
    Find the indices of the k best matches by cosine-similarities 
    between the tf-idf vectors
    '''
    real_search = preprocess_search(search_term)
    search_vec = transvector.transform([real_search])
    similarities = tfidf1.dot(search_vec.transpose())
    return np.argsort(similarities.toarray()[:,0])[-k:][::-1]
    

In [15]:

# Test our search
search_text = 'gun game platformer'
results = get_top_results(search_text,5)
print(results)
for i in results:
    print('###### Match ', i)
    print(desc_texts[i], '\n')
   

[ 514   20  577 1215   51]
###### Match  514
  metal guns   choose your gun and enjoy it  

###### Match  20
 epic guns enjoy the legendary arsenal of mr gun  beast outfits beast mode is on  coming soon explosive guns and new secret set of outfits 

###### Match  577
launch of turn undead 2 monster hunter the sequel to our turnbased puzzle platformer 

###### Match  1215
update 48 is here  halloween is coming to guns of boom get ready for a new actionpacked event  new spooky map  new fearinducing gun   new healing helmet  new ghastly animated costume  tons of bloodcurdling cosmetics to make you the spookiest gunslinger out there  share feature for your profile let the world know how tough you are at the push of a button  new looks customize your character even further with cool mustaches and more 

###### Match  51
new game 



In [16]:
search_text = 'strategy science fiction'
results = get_top_results(search_text,5)
print(results)
for i in results:
    print('###### Match ', i)
    print(desc_texts[i], '\n')

[ 853  673  824 1278  316]
###### Match  853
roulette strategy amp game 

###### Match  673
first release of onitama  features  full game  solo several difficulty levels vs artificial intelligence  pass and play unlock by connecting asmodee account  online mode sensei's path extension 20 new cards  customize boards with new music medieval and science fiction 

###### Match  824
to celebrate winning the queens enterprise award for innovation weve created two all new official scenarios  new ultimate board games scenario make a bestselling board game in this radically different scenario that has nothing to do with disease  new science denial scenario people around the world have stopped believing in science medicine and even diseases 

###### Match  1278
the wait is finally over welcome to armello on android  the king has fallen to a dark corruption and its up to you to stop him and your opponents for the good of armello choose your hero and step into the fray leverage subterfuge spells a